In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/seungheondoh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
client = MongoClient()
db = client.aboutdepart
database = list(db.Docdb.find())
testdf = pd.DataFrame(database)
testdf.head()

,Abstract,Author Keywords,Name,ResearchInterest,Title,_id,department,docs,text
0,[Entrepreneurs' decisions to exploit opportuni...,"[nan, Behavioral decision-making; Business gro...","Choi, Young Rok(최영록)","Entrepreneurship, Innovation Strategy, Family ...",[Entrepreneurs' decisions to exploit opportuni...,5bf7bdc420c66c03397b5abd,Graduate School of Technology and Innovation M...,[[Entrepreneurs' decisions to exploit opportun...,Entrepreneurs' decisions to exploit opportuni...
1,[Organizations use project teams to lower sear...,[expertise awareness; expertise use; knowledge...,"Hong, Woonki(홍운기)","Expertise utilization, Transactive menory syst...",[Explaining dyadic expertise use in knowledge ...,5bf7bdc520c66c03397b5abe,School of Business Administration(경영학부),[[Explaining dyadic expertise use in knowledge...,Explaining dyadic expertise use in knowledge ...
2,[This paper investigates information leakage f...,"[nan, nan, Business group; Chaebol; Control-ca...","Jung, Kooyul(정구열)","Accounting Information and capital market, Ana...",[Trading behavior prior to public release of a...,5bf7bdc520c66c03397b5abf,Graduate School of Technology and Innovation M...,[[Trading behavior prior to public release of ...,Trading behavior prior to public release of a...
3,[Studies on the effects of corporate social re...,[annual reports; communication; corporate soci...,"Kim, MinChung(김민중)",Marketing strategies and their impact on finan...,[CSR and Shareholder Value in the Restaurant I...,5bf7bdc520c66c03397b5ac0,School of Business Administration(경영학부),[[CSR and Shareholder Value in the Restaurant ...,CSR and Shareholder Value in the Restaurant I...
4,[Brand communities and corporate social respon...,[Brand community; Consumer communities; Corpor...,"Kim, Molan(김모란)","Online Database Marketing, Social Media Market...","[Consumer Communities Do Well, But Will They D...",5bf7bdc520c66c03397b5ac1,School of Business Administration(경영학부),"[[Consumer Communities Do Well, But Will They ...","Consumer Communities Do Well, But Will They D..."


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords


def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    
    docs = docs.lower()  # Convert to lowercase.
    docs = tokenizer.tokenize(docs)  # Split into words.
    docs = [w for w in docs if not w in stopwords.words('english')]
    
    # Remove numbers, but not words that contain numbers.
    docs = [token for token in docs if not token.isdigit()]
    
    # Remove words that are only one character.
    docs = [token for token in docs if len(token) > 1]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [lemmatizer.lemmatize(token) for token in docs]
  
    return docs

In [5]:
for idx, i in enumerate(testdf['text']):
    add = docs_preprocessor(i)
    testdf.iloc[idx]['text'] = add

In [6]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [7]:
zipdata = zip(testdf['Name'],testdf['text'])

In [8]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in zipdata]

In [9]:
tagged_data[3]

TaggedDocument(words=['csr', 'shareholder', 'value', 'restaurant', 'industry', 'role', 'csr', 'communication', 'annual', 'report', 'study', 'effect', 'corporate', 'social', 'responsibility', 'csr', 'shareholder', 'value', 'identified', 'csr', 'communication', 'marketing', 'channel', 'key', 'mechanism', 'necessary', 'translate', 'csr', 'shareholder', 'value', 'study', 'annual', 'report', 'considered', 'primary', 'information', 'source', 'used', 'firm', 'financial', 'stakeholder', 'investor', 'analyst', 'valuation', 'firm', 'specifically', 'study', 'examined', 'whether', 'extent', 'restaurant', 'firm', 'communicate', 'csr', 'annual', 'report', 'influence', 'effect', 'csr', 'shareholder', 'value', 'return', 'tobin', 'risk', 'equity', 'holder', 'risk', 'csr', 'disaggregated', 'two', 'distinct', 'type', 'primary', 'nonprimary', 'stakeholder', 'result', 'corroborate', 'communicating', 'csr', 'nonprimary', 'stakeholder', 'annual', 'report', 'help', 'nonprimary', 'stakeholder', 'csr', 'increas

In [10]:
import re 
import gensim
from gensim.models import doc2vec
from collections import namedtuple
from random import shuffle

In [33]:
model = gensim.models.Doc2Vec(vector_size=100, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=20)

#sample=1e-4, negative=5,

#shuffling is better (ot needed at each trianing epoch
shuffle(tagged_data)
#Build vocabulary from a sequence of sentences 
model.build_vocab(tagged_data)
#Update the model’s neural weights from a sequence of sentences
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

In [12]:
model.save("finalmodel")

In [13]:
model.wv.most_similar('pzk', topn=20)

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('fluorothieno', 0.9116326570510864),
 ('bithiophen', 0.8954272270202637),
 ('jsp', 0.8918386697769165),
 ('ferraro', 0.8901904821395874),
 ('thien', 0.8828878402709961),
 ('suppl', 0.8823558688163757),
 ('chls', 0.8801031112670898),
 ('germolo', 0.8791412115097046),
 ('kist', 0.8783347606658936),
 ('y1030', 0.8767693638801575),
 ('discov', 0.8756805658340454),
 ('5c6h6', 0.8748934268951416),
 ('butanolamine', 0.8729902505874634),
 ('ono', 0.8710193634033203),
 ('sytematic', 0.8679637908935547),
 ('τnπ', 0.8669286966323853),
 ('gaiii6l26', 0.8654748201370239),
 ('yoo', 0.8637548685073853),
 ('τππ', 0.860684335231781),
 ('cleansgin', 0.859939694404602)]

In [14]:
model.docvecs.most_similar('Kim, Chajoong(김차중)', topn=50)

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Self, James(SelfJames Andrew)', 0.7231125831604004),
 ('Baek, Joon Sang(백준상)(2011-07~2018-03)', 0.7183336615562439),
 ('Kim, KwanMyung(김관명)', 0.7142677903175354),
 ('Lee, Jacob C.(이채호)', 0.676101565361023),
 ('Kim, Molan(김모란)', 0.6481330394744873),
 ('Kyung, Gyouhyung(경규형)', 0.6373521089553833),
 ('Zhang, Lu(ZhangLu)', 0.6351935863494873),
 ('Kim, Yeolib(김여립)', 0.6206967830657959),
 ('Lim, Sunghoon(임성훈)', 0.6205252408981323),
 ('Jung, Dooyoung(정두영)', 0.6108125448226929),
 ('Lim, Chiehyeon(임치현)', 0.6104732155799866),
 ('Hong, Hwajung(홍화정)(2015-08~2018-03)', 0.6029444932937622),
 ('Kim, Duck Young(김덕영)', 0.5923179984092712),
 ('Kim, Young Choon(김영춘)', 0.5784534811973572),
 ('Kim, Jeongseob(김정섭)', 0.5741623640060425),
 ('Hong, Woonki(홍운기)', 0.5714349150657654),
 ('Park, Young-Woo(박영우)', 0.5601038932800293),
 ('Woo, Hangyun(우한균)', 0.5564560294151306),
 ('Chung, Jibum(정지범)', 0.5489473342895508),
 ('Hwang, Sung Ju(황성주)(2014-08~2018-01)', 0.5392651557922363),
 ('Lee, Changyong(이창용)', 0.536

In [49]:
docvec = model.docvecs.vectors_docs
docvec.shape

(258, 100)

In [54]:
wordvector = model.wv.vectors
type(wordvector)

numpy.ndarray

In [57]:
totalvector = np.vstack((docvec,wordvector))